# Mission to Mars - Web Scraping

In [80]:
# Import Splinter and BeautifulSoup
from splinter import Browser
from bs4 import BeautifulSoup #as soup
import pandas as pd 

In [64]:
# Set the executable path and initialize the chrome browser in SPLINTER
executable_path = {'executable_path': '../chromedriver'}
browser = Browser('chrome', **executable_path)

In [65]:
# Visit the mars nasa news site
url = 'https://mars.nasa.gov/news/'
browser.visit(url)

# Search for elements with a specific combination of tag (ul and li) and attribute (item_list and slide, respectively). 
# Tell browser to wait one second before searching for components. The optional delay is useful because sometimes dynamic pages take a little while to load, especially if they are image-heavy.
# Optional delay for loading the page
browser.is_element_present_by_css("ul.item_list li.slide", wait_time=1)

True

In [66]:
html = browser.html
news_soup = BeautifulSoup(html, 'html.parser')
# Get our parent element
#  . is used for selecting classes, such as item_list, so 'ul.item_list li.slide' pinpoints the <li /> tag with the class of slide 
# and the <ul /> tag with a class of item_list. 
# CSS works from right to left, such as returning the last item on the list instead of the first
# Because of this, when using select_one, the first matching element returned will be a <li /> element with a class of slide and all nested elements within it
slide_elem = news_soup.select_one('ul.item_list li.slide')

In [67]:
# Looking for the title
#page > div.grid_list_page.module.content_page > div > article > div > section > div > ul > li:nth-child(1) > div > div > div.content_title
# Looking for the body
#page > div.grid_list_page.module.content_page > div > article > div > section > div > ul > li:nth-child(1) > div > div > div.article_teaser_body

In [68]:
# IMPORTANT
# There are two methods used to find tags and attributes with BeautifulSoup:
# .find() is used when we want only the first class and attribute we’ve specified.
# .find_all() is used when we want to retrieve all of the tags and attributes.

In [69]:
slide_elem.find("div", class_='content_title')

<div class="content_title"><a href="/news/8723/nasas-perseverance-rover-will-carry-first-spacesuit-materials-to-mars/" target="_self">NASA's Perseverance Rover Will Carry First Spacesuit Materials to Mars</a></div>

In [70]:
# Use the parent element to find the first `a` tag and save it as `news_title`
news_title = slide_elem.find("div", class_='content_title').get_text()
news_title

"NASA's Perseverance Rover Will Carry First Spacesuit Materials to Mars"

In [71]:
slide_elem.find("div", class_='article_teaser_body')

<div class="article_teaser_body">In a Q&amp;A, spacesuit designer Amy Ross explains how five samples, including a piece of helmet visor, will be tested aboard the rover, which is targeting a July 30 launch. </div>

In [72]:
news_summary = slide_elem.find("div", class_='article_teaser_body').get_text()
news_summary

'In a Q&A, spacesuit designer Amy Ross explains how five samples, including a piece of helmet visor, will be tested aboard the rover, which is targeting a July 30 launch. '

### Featured Images

In [73]:
# Visit URL
url = 'https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars'
browser.visit(url)

In [74]:
# id=“full_image”, in HTML an id is a completely unique identifier
# Find and click the full image button
full_image_elem = browser.find_by_id('full_image')
full_image_elem.click()

In [ ]:
# Find the more info button and click that
browser.is_element_present_by_text('more info', wait_time=1)
more_info_elem = browser.links.find_by_partial_text('more info')
more_info_elem.click()

In [77]:
# Parse the resulting html with soup
html = browser.html
img_soup = soup(html, 'html.parser')

In [78]:
# Find the relative image url
# figure.lede references the <figure /> tag and its class, lede
# a is the next tag nested inside the <figure /> tag
# An img tag is also nested within this HTML, so we’ve included that as well
# .get("src") pulls the link to the image.
img_url_rel = img_soup.select_one('figure.lede a img').get("src")
img_url_rel

'/spaceimages/images/largesize/PIA22893_hires.jpg'

In [79]:
# Use the base URL to create an absolute URL
img_url = f'https://www.jpl.nasa.gov{img_url_rel}'
img_url

'https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA22893_hires.jpg'

In [84]:
# Create a new DataFrame from the HTML table - a tidy, Pandas-friendly representation of the HTML table we were just viewing on the website
# - The Pandas function read_html() specifically searches for and returns a list of tables found in the HTML
# index 0 - pull only the first table (or the first item in the list)
# Turn the table into a DataFrame
# Assign columns to the new DataFrame for additional clarity
# Turn the Description column into the DataFrame’s index. inplace=True - updated index remains in place (does NOT reassign the DataFrame to a new variable)

df = pd.read_html('http://space-facts.com/mars/')[0]
df.columns=['description', 'value']
df.set_index('description', inplace=True)
df.to_html()

'<table border="1" class="dataframe">\n  <thead>\n    <tr style="text-align: right;">\n      <th></th>\n      <th>value</th>\n    </tr>\n    <tr>\n      <th>description</th>\n      <th></th>\n    </tr>\n  </thead>\n  <tbody>\n    <tr>\n      <th>Equatorial Diameter:</th>\n      <td>6,792 km</td>\n    </tr>\n    <tr>\n      <th>Polar Diameter:</th>\n      <td>6,752 km</td>\n    </tr>\n    <tr>\n      <th>Mass:</th>\n      <td>6.39 × 10^23 kg (0.11 Earths)</td>\n    </tr>\n    <tr>\n      <th>Moons:</th>\n      <td>2 (Phobos &amp; Deimos)</td>\n    </tr>\n    <tr>\n      <th>Orbit Distance:</th>\n      <td>227,943,824 km (1.38 AU)</td>\n    </tr>\n    <tr>\n      <th>Orbit Period:</th>\n      <td>687 days (1.9 years)</td>\n    </tr>\n    <tr>\n      <th>Surface Temperature:</th>\n      <td>-87 to -5 °C</td>\n    </tr>\n    <tr>\n      <th>First Record:</th>\n      <td>2nd millennium BC</td>\n    </tr>\n    <tr>\n      <th>Recorded By:</th>\n      <td>Egyptian astronomers</td>\n    </tr>\

In [85]:
# End the browser session
browser.quit()